In [ ]:
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm
import ast
from time import sleep
from bs4 import BeautifulSoup
#change file location according to where this notebook is located.
data_folder = '/content/drive/MyDrive/pop_pharmacogenomics/onsides_intl/uk_drug_labels/'
api = 'bbf40496-e20e-4f79-9636-b63d4a3deea7'

This notebook should iterate through the medicines.org.uk/emc website to download the html drug label files into a sub-folder "raw/" in the data_folder above.

## 1. get list of all drugs

In [ ]:
ingredient_list = []
for l in tqdm(['0-9', '', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']):
  url = 'https://www.medicines.org.uk/emc/browse-ingredients/{}'.format(l)
  r = requests.get(url).text
  s = BeautifulSoup(r)
  drugs = s.find(class_='browse-results')
  for d in drugs.find_all('a', href=True):
    drug_id = d['href']
    drug_name = d.text
    ingredient_list.append([drug_id, drug_name])

100%|██████████| 27/27 [00:20<00:00,  1.32it/s]


In [ ]:
ingredient_df = pd.DataFrame(ingredient_list, columns = ['ingredient_id', 'ingredient_name'])
ingredient_df.to_csv(data_folder+'ingredient_data.csv', index=False)
ingredient_df.head(1)

OSError: ignored

In [ ]:
product_list = []
redo = []
for ingredient_id, ingredient_name in tqdm(zip(ingredient_df.ingredient_id, ingredient_df.ingredient_name)):
  url = 'https://www.medicines.org.uk{}?offset=1&limit=200'.format(ingredient_id)
  r = requests.get(url).text
  s = BeautifulSoup(r)
  #number of items
  item_num = s.find(class_='search-results-header').find('span').text.replace(' products found','').replace(' product found','')
  #make table of items
  product_data = s.find_all(class_='search-results-product')
  for i in product_data:
    product_id = i.find(class_='search-results-product-info-title-link emc-link', href=True)['href']
    product_name = i.find(class_='search-results-product-info-title-link emc-link').text
    active_ingredients = i.find(class_="search-results-product-info-type emc-text-default").text
    company_name = i.find(class_="search-results-product-info-company").text
    product_list.append([ingredient_id, ingredient_name, product_id, product_name, active_ingredients, company_name])
  if int(item_num) > 200:
    redo.append([ingredient_id, ingredient_name])

2021it [13:53,  2.43it/s]


In [ ]:
drug_df = pd.DataFrame(product_list, columns = ['ingredient_id', 'ingredient_name', 'product_id', 'product_name', 'active_ingredients', 'company_name'])
drug_df.to_csv(data_folder+'drug_data.csv', index=False)
drug_df.head(1)

,ingredient_id,ingredient_name,product_id,product_name,active_ingredients,company_name
0,/emc/ingredient/1524,"2,4-dichlorobenzyl alcohol",/emc/product/1589/smpc,\n Boots Antiseptic Cream\n...,"allantoin, cetrimide, 2,4-dichlorobenzyl alcohol",THE BOOTS COMPANY PLC


In [ ]:
print(redo)

NameError: ignored

## 2. use list of drugs to get SmPC pages parsed from Website


In [ ]:
drug_df = pd.read_csv(data_folder+'drug_data.csv')
drug_df.head(1)

In [ ]:
for product in tqdm(drug_df.product_id.unique().tolist()):
  url = 'https://www.medicines.org.uk{}'.format(product)
  r = requests.get(url)
  #save raw html
  f = open(data_folder+'raw/{}.txt'.format(product.split('/')[-2]), 'w')
  f.write(r.text)
  f.close()

In [ ]:
redo = []
for product in tqdm(drug_df.product_id.unique().tolist()):
  f = (data_folder+'raw/{}.txt'.format(product.split('/')[-2]))
  try:
    with open(f) as fi:
      s = BeautifulSoup(fi, 'html.parser')
    if s.find('title').text == 'emc - are you human?':
      ##retry
      url = 'https://www.medicines.org.uk{}'.format(product)
      r = requests.get(url)
      s = BeautifulSoup(r.text, 'html.parser')
      if s.find('title').text != 'emc - are you human?':
        #save raw html
        f = open(data_folder+'raw/{}.txt'.format(product.split('/')[-2]), 'w')
        f.write(r.text)
        f.close()
        redo.append([product, 'done'])
      else:
        redo.append([product, 'failed'])
  except:
    ##retry
    url = 'https://www.medicines.org.uk{}'.format(product)
    r = requests.get(url)
    s = BeautifulSoup(r.text, 'html.parser')
    if s.find('title').text != 'emc - are you human?':
      #save raw html
      f = open(data_folder+'raw/{}.txt'.format(product.split('/')[-2]), 'w')
      f.write(r.text)
      f.close()
      redo.append([product, 'done'])
    else:
      redo.append([product, 'failed'])
  sleep(1)

 80%|████████  | 7480/9334 [3:27:01<53:40,  1.74s/it]